In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [5]:
from pathlib import Path

# Configurable parts
SOURCE = "dukascopy"
SYMBOL = "USDJPY"
MINUTES = 1
EVENT = '58m-dollar'
START_DATE = "20210101"
END_DATE = "20241231"

# Build base name
RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"
# RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{EVENT}-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}-processed.pkl"

In [6]:
df = pd.read_pickle(PROCESSED_FILE_PATH)
df.head()

,open,high,low,close,volume,spread,close_delta,close_return,close_log_return,ret_mean_5,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-04 12:40:00,102.9310,102.9360,102.9015,102.9045,4.435300e+11,0.002134,-0.0270,-0.000262,-0.000262,0.000017,...,4,1,1.224647e-16,-1.0,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 12:41:00,102.9040,102.9125,102.8875,102.8915,2.647700e+11,0.002636,-0.0130,-0.000126,-0.000126,-0.000028,...,4,1,1.224647e-16,-1.0,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 12:42:00,102.8910,102.8945,102.8840,102.8910,2.431300e+11,0.002603,-0.0005,-0.000005,-0.000005,-0.000067,...,4,1,1.224647e-16,-1.0,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 12:43:00,102.8920,102.8920,102.8865,102.8890,9.203000e+10,0.003040,-0.0020,-0.000019,-0.000019,-0.000077,...,4,1,1.224647e-16,-1.0,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 12:44:00,102.8875,102.9025,102.8875,102.9025,9.773000e+10,0.002312,0.0135,0.000131,0.000131,-0.000056,...,4,1,1.224647e-16,-1.0,0.0,1.0,0.724793,0.688967,0.5,0.866025


### Reset Timestamp into Column

In [7]:
df = df.reset_index()

In [7]:
df.head()

,timestamp,open,high,low,close,volume,spread,close_delta,close_return,close_log_return,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
0,2021-01-03 23:44:00,103.2920,103.3005,103.2875,103.2890,1.147400e+11,0.003725,-0.0030,-0.000029,-0.000029,...,3,1,-0.258819,0.965926,-0.781831,0.62349,0.571268,0.820763,0.5,0.866025
1,2021-01-03 23:45:00,103.2860,103.3090,103.2835,103.3040,2.637000e+11,0.004039,0.0150,0.000145,0.000145,...,3,1,-0.258819,0.965926,-0.781831,0.62349,0.571268,0.820763,0.5,0.866025
2,2021-01-03 23:46:00,103.3045,103.3180,103.2980,103.2980,1.622600e+11,0.003704,-0.0060,-0.000058,-0.000058,...,3,1,-0.258819,0.965926,-0.781831,0.62349,0.571268,0.820763,0.5,0.866025
3,2021-01-03 23:47:00,103.3010,103.3015,103.2805,103.2895,1.991100e+11,0.003133,-0.0085,-0.000082,-0.000082,...,3,1,-0.258819,0.965926,-0.781831,0.62349,0.571268,0.820763,0.5,0.866025
4,2021-01-03 23:48:00,103.2855,103.2855,103.2690,103.2825,1.275100e+11,0.002381,-0.0070,-0.000068,-0.000068,...,3,1,-0.258819,0.965926,-0.781831,0.62349,0.571268,0.820763,0.5,0.866025


# Techinical Indicator

In [9]:
import numpy as np
import pandas_ta as pdta
from ta.volatility import AverageTrueRange, BollingerBands, DonchianChannel
from ta.momentum import StochasticOscillator, RSIIndicator
from ta.trend import MACD, EMAIndicator, SMAIndicator

In [25]:
print(df.ta.indicators())


Pandas TA - Technical Analysis Indicators - v0.4.71b0

Indicators and Utilities [154]:
    aberration, accbands, ad, adosc, adx, alligator, alma, alphatrend, amat, ao, aobv, apo, aroon, atr, atrts, bbands, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chandelier_exit, chop, cksp, cmf, cmo, coppock, crsi, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, exhc, fisher, fwma, ha, hilo, hl2, hlc3, hma, ht_trendline, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, mama, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, pivots, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, reflex, rma, roc, rsi, rsx, rvgi, rvi, rwi, short_run, sinwma, skew, slope, sma, smc, smi, smma, squeeze, squeeze_pro, ssf, ssf3, stc, stdev, stoch, stochf, stochrsi, supertrend, swma, t3, tema, thermo, tmo, tos_stdevall, trendflex

### EMA & SMA

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
1486413   NaN
1486414   NaN
1486415   NaN
1486416   NaN
1486417   NaN
Name: EMA_10, Length: 2972836, dtype: float64

In [10]:
# 5-period EMA
ema5 = EMAIndicator(close=df['close'], window=5)
df['ema5'] = ema5.ema_indicator()
ema5_slope = df['ema5'].diff()
df['ema5_slope'] = ema5_slope
# 20-period EMA
ema20 = EMAIndicator(close=df['close'], window=20)
df['ema20'] = ema20.ema_indicator()
ema20_slope = df['ema20'].diff()
df['ema20_slope'] = ema20_slope

In [12]:
pdta.ema(df.close, window=5)

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
              ...    
1486413    157.004414
1486414    157.004975
1486415    157.005343
1486416    157.004645
1486417    157.005346
Name: EMA_10, Length: 1486418, dtype: float64

### ATR

In [17]:
df['atr14'] = pdta.atr(
    df.high,
    df.low,
    df.close,
    length=14,
)

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
             ...   
1486413    0.013912
1486414    0.013025
1486415    0.013488
1486416    0.013096
1486417    0.012910
Name: ATRr_14, Length: 1486418, dtype: float64

In [18]:
df['atr20'] = pdta.atr(
    df.high,
    df.low,
    df.close,
    length=20,
)

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
             ...   
1486413    0.014480
1486414    0.013831
1486415    0.014114
1486416    0.013808
1486417    0.013643
Name: ATRr_20, Length: 1486418, dtype: float64

In [32]:
df['close_to_atr'] = df['close_delta'] / df['atr20']

### BollingerBands

In [19]:
help(pdta.bbands)

Help on function bbands in module pandas_ta.volatility.bbands:

bbands(close: pandas.core.series.Series, length: Union[int, numpy.integer] = None, lower_std: Union[int, numpy.integer, float, numpy.floating] = None, upper_std: Union[int, numpy.integer, float, numpy.floating] = None, ddof: Union[int, numpy.integer] = None, mamode: str = None, talib: bool = None, offset: Union[int, numpy.integer] = None, **kwargs: Optional[dict]) -> pandas.core.frame.DataFrame
    Bollinger Bands

    This indicator, by John Bollinger, attempts to quantify volatility by
    creating lower and upper bands centered around a moving average.

    Sources:
        * [tradingview](https://www.tradingview.com/wiki/Bollinger_Bands_(BB))

    Parameters:
        close (Series): ```close``` Series
        length (int): The period. Default: ```5```
        lower_std (IntFloat): Lower standard deviation. Default: ```2.0```
        upper_std (IntFloat): Upper standard deviation. Default: ```2.0```
        ddof (int): 

In [23]:
df = pd.concat([
    df,
    pdta.bbands(
        df.close,
        length=5,
    )
])

In [ ]:
bb = BollingerBands(
    close=df['close'],
    window=20,
    window_dev=2
)
df['bb_upper'] = bb.bollinger_hband()
df['bb_lower'] = bb.bollinger_lband()
df['bb_mavg'] = bb.bollinger_mavg()
df['bb_width'] = df['bb_upper'] - df['bb_lower']
df['bb_position'] = (df['close'] - df['bb_lower']) / df['bb_width']

### Donchian Channel

In [ ]:
dc = DonchianChannel(
    high=df['high'],
    low=df['low'],
    close=df['close'],
    window=20
)
df['donchian_upper'] = dc.donchian_channel_hband()
df['donchian_lower'] = dc.donchian_channel_lband()
df['donchian_mid'] = dc.donchian_channel_mband()
df['donchian_width'] = df['donchian_upper'] - df['donchian_lower']

### Stochastic Oscillator

In [ ]:
stoch = StochasticOscillator(
    high=df['high'],
    low=df['low'],
    close=df['close'],
    window=14,
    smooth_window=3
)
df['stoch_k'] = stoch.stoch()
df['stoch_d'] = stoch.stoch_signal()

### RSI (Relative Strength Index)

In [22]:
rsi = RSIIndicator(close=df['close'], window=14)
df['rsi14'] = rsi.rsi()
df['rsi14_slope'] = df['rsi14'].diff()

In [ ]:
df['rsi14']

### MACD

In [ ]:
macd = MACD(close=df['close'], window_slow=26, window_fast=12, window_sign=9)
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()
df['macd_diff'] = macd.macd_diff()

### ADX

In [ ]:
from ta.trend import ADXIndicator
# 建立 ADX 指標
adx14 = ADXIndicator(
    high=df["high"], 
    low=df["low"], 
    close=df["close"], 
    window=14,         # 與 ta-lib 的 timeperiod 對應
    fillna=False
)

# ADX 值
df["adx14"] = adx14.adx()

# +DI / -DI
df["plus_di14"] = adx14.adx_pos()
df["minus_di14"] = adx14.adx_neg()

### Pandas TA

In [8]:
pdta.adx(    
    high=df["high"], 
    low=df["low"], 
    close=df["close"], 
    window=14,  
)

,ADX_14,ADXR_14_2,DMP_14,DMN_14
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
1486413,23.224865,24.001608,0.043640,0.057263
1486414,22.530317,23.251579,0.040523,0.053173
1486415,22.920023,23.072444,0.037628,0.066875
1486416,23.281893,22.906105,0.034941,0.062098


# Time Features

In [ ]:
df['unix_time'] = df['timestamp'].astype('int64') / 1e9

In [ ]:
df['hour'] = df['timestamp'].dt.hour
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

In [ ]:
df['dow'] = df['timestamp'].dt.dayofweek  # Monday=0
df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 7)

In [ ]:
df['dom'] = df['timestamp'].dt.day
df['dom_sin'] = np.sin(2 * np.pi * df['dom'] / 31)
df['dom_cos'] = np.cos(2 * np.pi * df['dom'] / 31)

In [ ]:
df['month'] = df['timestamp'].dt.month
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

## Drop NaN

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()
df.isna().sum()

In [ ]:
df = df.set_index('timestamp')

In [ ]:
df.head()

In [ ]:
df.columns

# Saving the file

In [ ]:
df.to_pickle(PROCESSED_FILE_PATH)